In [1]:
import pandas as pd
import numpy as np
import talib
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

C:\ProgramData\Anaconda3\lib\site-packages\ffn\core.py:27: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)


In [2]:
%matplotlib inline

In [3]:
#Read the Nifty 200 stocks historical data from 2007 to 2020
stock_prices_data = pd.read_pickle('yfinance_nifty_200_stocks_price_data_2007.pkl')
nifty_200 = pd.read_csv('ind_nifty200list.csv')

In [4]:
#Keep data till 2020 only as we only consider month end data
stock_prices_data = stock_prices_data.loc[stock_prices_data.index.year<=2020,:]

In [5]:
#Sample the daily data at Monthly Frequency
stock_prices_data = stock_prices_data.groupby('Symbol').resample('BM').first()
del stock_prices_data['Symbol']
stock_prices_data = stock_prices_data.reset_index().set_index('Date')

In [6]:
stock_prices_data.head()

,Symbol,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2007-01-31,AARTIIND,14.875,15.20,14.225,14.300,5.804401,203876.0
2007-02-28,AARTIIND,15.250,15.50,14.300,14.550,5.905878,165432.0
2007-03-30,AARTIIND,13.200,13.25,12.525,12.925,5.445386,57136.0
2007-04-30,AARTIIND,11.575,11.85,11.500,11.575,4.999304,49228.0
2007-05-31,AARTIIND,13.775,14.00,13.575,13.825,5.971091,44470.0


In [7]:
#Following Calculates monthly Return for Each Stock in Nifty 200
stock_prices_data['1MonthReturn'] = stock_prices_data.groupby('Symbol')['Adj Close'].pct_change(1)
stock_prices_data['1MonthReturnCum'] = stock_prices_data['1MonthReturn'] + 1

In [8]:
stock_prices_data.head()

,Symbol,Open,High,Low,Close,Adj Close,Volume,1MonthReturn,1MonthReturnCum
Date,,,,,,,,,
2007-01-31,AARTIIND,14.875,15.20,14.225,14.300,5.804401,203876.0,NaN,NaN
2007-02-28,AARTIIND,15.250,15.50,14.300,14.550,5.905878,165432.0,0.017483,1.017483
2007-03-30,AARTIIND,13.200,13.25,12.525,12.925,5.445386,57136.0,-0.077972,0.922028
2007-04-30,AARTIIND,11.575,11.85,11.500,11.575,4.999304,49228.0,-0.081919,0.918081
2007-05-31,AARTIIND,13.775,14.00,13.575,13.825,5.971091,44470.0,0.194384,1.194384


In [9]:
#forward_holding_period = 12 Months or 1 Year
forward_holding_period = 12
new_df = stock_prices_data.iloc[::-1].groupby('Symbol').rolling(window=forward_holding_period)['1MonthReturnCum']\
.apply(np.prod, raw=True).reset_index().set_index(['Date', 'Symbol']).rename(columns={'1MonthReturnCum' : 'Forward_Holding_PeriodReturn'})
stock_prices_data = stock_prices_data.reset_index().set_index(['Date', 'Symbol']).join(new_df)
stock_prices_data.reset_index(inplace=True)
stock_prices_data.set_index('Date', inplace=True)

In [10]:
# Calculate standard deviation of monthly returns for forward period to be used in calculating risk adjusted or normalized
# returns
new_df = stock_prices_data.iloc[::-1].groupby('Symbol').rolling(window=forward_holding_period)['1MonthReturn']\
.std().reset_index().set_index(['Date', 'Symbol']).rename(columns={'1MonthReturn' : 'Forward_Std'})
stock_prices_data = stock_prices_data.reset_index().set_index(['Date', 'Symbol']).join(new_df)
stock_prices_data.reset_index(inplace=True)
stock_prices_data.set_index('Date', inplace=True)
stock_prices_data['Forward_Std'] = stock_prices_data['Forward_Std']*np.sqrt(12)

In [11]:
stock_prices_data['Forward_NormRet'] = (stock_prices_data['Forward_Holding_PeriodReturn'] - 1) / stock_prices_data['Forward_Std']

In [12]:
stock_prices_data.loc['2020-01-31']

,Symbol,Open,High,Low,Close,Adj Close,Volume,1MonthReturn,1MonthReturnCum,Forward_Holding_PeriodReturn,Forward_Std,Forward_NormRet
Date,,,,,,,,,,,,
2020-01-31,AARTIIND,836.000000,843.000000,831.099976,837.799988,833.868042,87969.0,0.012753,1.012753,1.405144,0.499250,0.811506
2020-01-31,ABBOTINDIA,13170.599609,13452.200195,13080.000000,13140.000000,12919.087891,4037.0,0.041105,1.041105,1.240921,0.299887,0.803373
2020-01-31,ABCAPITAL,103.199997,103.400002,101.199997,101.500000,101.500000,982667.0,-0.052730,0.947270,0.873542,0.774266,-0.163327
2020-01-31,ABFRL,232.899994,236.850006,230.149994,234.100006,234.100006,241005.0,0.032870,1.032870,0.711891,0.563234,-0.511527
2020-01-31,ACC,1452.500000,1457.000000,1438.199951,1440.099976,1401.717285,321092.0,-0.059557,0.940443,1.172704,0.427236,0.404237
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-31,VOLTAS,659.299988,663.000000,651.349976,653.299988,644.731689,820283.0,-0.060135,0.939865,1.158191,0.402445,0.393075
2020-01-31,WHIRLPOOL,2386.000000,2387.250000,2325.000000,2333.250000,2322.149658,32427.0,0.080659,1.080659,1.014786,0.291885,0.050656
2020-01-31,WIPRO,246.500000,248.649994,246.199997,247.699997,246.704010,1413866.0,0.038139,1.038139,1.484805,0.392152,1.236269


In [13]:
#Calculate the median return
median_universe_return_df = pd.DataFrame(stock_prices_data.groupby('Date')['Forward_NormRet'].median())
median_universe_return_df = median_universe_return_df.rename(columns={'Forward_NormRet': 'MedianReturn'})

# Use median return to label each stock as buy = 1 if its 12M Forward Return >= Median of all stocks on that date
# else label it as sell = 0
stock_prices_data = stock_prices_data.join(median_universe_return_df)
stock_prices_data['Target_Var'] = np.select([stock_prices_data['Forward_NormRet'] >= stock_prices_data['MedianReturn']],[1], default=0)

In [14]:
stock_prices_data.loc['2020-01-31']

,Symbol,Open,High,Low,Close,Adj Close,Volume,1MonthReturn,1MonthReturnCum,Forward_Holding_PeriodReturn,Forward_Std,Forward_NormRet,MedianReturn,Target_Var
Date,,,,,,,,,,,,,,
2020-01-31,AARTIIND,836.000000,843.000000,831.099976,837.799988,833.868042,87969.0,0.012753,1.012753,1.405144,0.499250,0.811506,0.262704,1
2020-01-31,ABBOTINDIA,13170.599609,13452.200195,13080.000000,13140.000000,12919.087891,4037.0,0.041105,1.041105,1.240921,0.299887,0.803373,0.262704,1
2020-01-31,ABCAPITAL,103.199997,103.400002,101.199997,101.500000,101.500000,982667.0,-0.052730,0.947270,0.873542,0.774266,-0.163327,0.262704,0
2020-01-31,ABFRL,232.899994,236.850006,230.149994,234.100006,234.100006,241005.0,0.032870,1.032870,0.711891,0.563234,-0.511527,0.262704,0
2020-01-31,ACC,1452.500000,1457.000000,1438.199951,1440.099976,1401.717285,321092.0,-0.059557,0.940443,1.172704,0.427236,0.404237,0.262704,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-31,VOLTAS,659.299988,663.000000,651.349976,653.299988,644.731689,820283.0,-0.060135,0.939865,1.158191,0.402445,0.393075,0.262704,1
2020-01-31,WHIRLPOOL,2386.000000,2387.250000,2325.000000,2333.250000,2322.149658,32427.0,0.080659,1.080659,1.014786,0.291885,0.050656,0.262704,0
2020-01-31,WIPRO,246.500000,248.649994,246.199997,247.699997,246.704010,1413866.0,0.038139,1.038139,1.484805,0.392152,1.236269,0.262704,1


In [15]:
# Calculate CCI Indicator over different periods
for period in [6,12,24,36, 48, 60]:
    new_df = pd.DataFrame(stock_prices_data.groupby('Symbol').apply(
        lambda x: talib.CCI(x.High, x.Low, x.Close, period))).reset_index().rename(
        columns={0: 'CCI_' + str(period) +'M'}).set_index(
        ['Date', 'Symbol'])

    stock_prices_data = stock_prices_data.reset_index().set_index(['Date', 'Symbol']).join(new_df)
    stock_prices_data.reset_index(inplace=True)
    stock_prices_data.set_index('Date', inplace=True)

# Calculate RSI Indicator over different periods
for period in [6,12,24,36, 48, 60]:
    new_df = pd.DataFrame(stock_prices_data.groupby('Symbol').apply(
        lambda x: talib.RSI(x['Adj Close'], period))).reset_index().rename(
        columns={0: 'RSI_' + str(period) +'M'}).set_index(
        ['Date', 'Symbol'])

    stock_prices_data = stock_prices_data.reset_index().set_index(['Date', 'Symbol']).join(new_df)
    stock_prices_data.reset_index(inplace=True)
    stock_prices_data.set_index('Date', inplace=True)
    
# Calculate ROC Indicator over different periods
for period in [6,12,24,36, 48, 60]:
    new_df = pd.DataFrame(stock_prices_data.groupby('Symbol').apply(
        lambda x: talib.ROC(x['Adj Close'], period))).reset_index().rename(
        columns={0: 'ROC_' + str(period) +'M'}).set_index(
        ['Date', 'Symbol'])

    stock_prices_data = stock_prices_data.reset_index().set_index(['Date', 'Symbol']).join(new_df)
    stock_prices_data.reset_index(inplace=True)
    stock_prices_data.set_index('Date', inplace=True)
    
# Calculate STDDEV Indicator over different periods
for period in [6,12,24,36, 48, 60]:
    new_df = pd.DataFrame(stock_prices_data.groupby('Symbol').apply(
        lambda x: talib.STDDEV(x['1MonthReturn'], period, 1))).reset_index().rename(
        columns={0: 'STDDEV_' + str(period) +'M'}).set_index(
        ['Date', 'Symbol'])

    stock_prices_data = stock_prices_data.reset_index().set_index(['Date', 'Symbol']).join(new_df)
    stock_prices_data.reset_index(inplace=True)
    stock_prices_data.set_index('Date', inplace=True)

In [23]:
# Calculate OBV
new_df = pd.DataFrame(stock_prices_data.groupby('Symbol').apply(
        lambda x: talib.OBV(x.Close, x.Volume))).reset_index().rename(
        columns={0: 'OBV'}).set_index(
        ['Date', 'Symbol'])


stock_prices_data = stock_prices_data.reset_index().set_index(['Date', 'Symbol']).join(new_df)
stock_prices_data.reset_index(inplace=True)
stock_prices_data.set_index('Date', inplace=True)

In [24]:
stock_prices_data.loc['2020-01-31']

,Symbol,Open,High,Low,Close,Adj Close,Volume,1MonthReturn,1MonthReturnCum,Forward_Holding_PeriodReturn,...,ROC_36M,ROC_48M,ROC_60M,STDDEV_6M,STDDEV_12M,STDDEV_24M,STDDEV_36M,STDDEV_48M,STDDEV_60M,OBV
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-31,AARTIIND,836.000000,843.000000,831.099976,837.799988,833.868042,87969.0,0.012753,1.012753,1.405144,...,143.220558,229.647283,599.637797,0.077024,0.096529,0.093557,0.089840,0.089051,0.099714,1.128683e+07
2020-01-31,ABBOTINDIA,13170.599609,13452.200195,13080.000000,13140.000000,12919.087891,4037.0,0.041105,1.041105,1.240921,...,182.576396,127.689620,255.536450,0.062150,0.068494,0.065245,0.070296,0.068000,0.072357,2.918290e+05
2020-01-31,ABCAPITAL,103.199997,103.400002,101.199997,101.500000,101.500000,982667.0,-0.052730,0.947270,0.873542,...,NaN,NaN,NaN,0.131651,0.129005,0.111584,NaN,NaN,NaN,-3.909298e+06
2020-01-31,ABFRL,232.899994,236.850006,230.149994,234.100006,234.100006,241005.0,0.032870,1.032870,0.711891,...,70.069028,-5.604836,95.246041,0.070840,0.060736,0.085669,0.077991,0.094914,0.110052,1.156718e+07
2020-01-31,ACC,1452.500000,1457.000000,1438.199951,1440.099976,1401.717285,321092.0,-0.059557,0.940443,1.172704,...,12.548327,12.018352,11.771332,0.046849,0.064782,0.072977,0.072625,0.072104,0.070968,9.482860e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-31,VOLTAS,659.299988,663.000000,651.349976,653.299988,644.731689,820283.0,-0.060135,0.939865,1.158191,...,105.820513,113.313942,184.813746,0.052185,0.054493,0.064099,0.074659,0.090380,0.090200,7.036112e+07
2020-01-31,WHIRLPOOL,2386.000000,2387.250000,2325.000000,2333.250000,2322.149658,32427.0,0.080659,1.080659,1.014786,...,164.633710,263.965466,261.988647,0.097033,0.087956,0.093322,0.086561,0.085406,0.081788,2.064536e+06
2020-01-31,WIPRO,246.500000,248.649994,246.199997,247.699997,246.704010,1413866.0,0.038139,1.038139,1.484805,...,42.599223,24.434436,32.606942,0.069109,0.077292,0.065034,0.064173,0.061147,0.063949,-2.959860e+07


In [25]:
def MLModel(X, y, max_depth, n_estimators):
    clf = RandomForestClassifier(max_depth=max_depth, random_state=0,n_estimators=n_estimators)
    clf.fit(X,y)
    return clf

In [28]:
estimation_window = 60 # 5 years
forward_return_horizon = 12 # 1 year
rebalance_frequency = 1 # Monthly

In [32]:
dates = np.unique(stock_prices_data.index)

In [67]:
model_training_end_dates = dates[estimation_window:-forward_return_horizon]
model_training_start_dates = dates[0:-estimation_window-forward_return_horizon]
rebalancing_dates = dates[estimation_window+forward_return_horizon:]

In [238]:
len(model_training_end_dates)

96

In [239]:
len(model_training_start_dates)

96

In [240]:
len(rebalancing_dates)

96

In [68]:
model_training_start_dates

array(['2007-01-31T00:00:00.000000000', '2007-02-28T00:00:00.000000000',
       '2007-03-30T00:00:00.000000000', '2007-04-30T00:00:00.000000000',
       '2007-05-31T00:00:00.000000000', '2007-06-29T00:00:00.000000000',
       '2007-07-31T00:00:00.000000000', '2007-08-31T00:00:00.000000000',
       '2007-09-28T00:00:00.000000000', '2007-10-31T00:00:00.000000000',
       '2007-11-30T00:00:00.000000000', '2007-12-31T00:00:00.000000000',
       '2008-01-31T00:00:00.000000000', '2008-02-29T00:00:00.000000000',
       '2008-03-31T00:00:00.000000000', '2008-04-30T00:00:00.000000000',
       '2008-05-30T00:00:00.000000000', '2008-06-30T00:00:00.000000000',
       '2008-07-31T00:00:00.000000000', '2008-08-29T00:00:00.000000000',
       '2008-09-30T00:00:00.000000000', '2008-10-31T00:00:00.000000000',
       '2008-11-28T00:00:00.000000000', '2008-12-31T00:00:00.000000000',
       '2009-01-30T00:00:00.000000000', '2009-02-27T00:00:00.000000000',
       '2009-03-31T00:00:00.000000000', '2009-04-30

In [69]:
model_training_end_dates

array(['2012-01-31T00:00:00.000000000', '2012-02-29T00:00:00.000000000',
       '2012-03-30T00:00:00.000000000', '2012-04-30T00:00:00.000000000',
       '2012-05-31T00:00:00.000000000', '2012-06-29T00:00:00.000000000',
       '2012-07-31T00:00:00.000000000', '2012-08-31T00:00:00.000000000',
       '2012-09-28T00:00:00.000000000', '2012-10-31T00:00:00.000000000',
       '2012-11-30T00:00:00.000000000', '2012-12-31T00:00:00.000000000',
       '2013-01-31T00:00:00.000000000', '2013-02-28T00:00:00.000000000',
       '2013-03-29T00:00:00.000000000', '2013-04-30T00:00:00.000000000',
       '2013-05-31T00:00:00.000000000', '2013-06-28T00:00:00.000000000',
       '2013-07-31T00:00:00.000000000', '2013-08-30T00:00:00.000000000',
       '2013-09-30T00:00:00.000000000', '2013-10-31T00:00:00.000000000',
       '2013-11-29T00:00:00.000000000', '2013-12-31T00:00:00.000000000',
       '2014-01-31T00:00:00.000000000', '2014-02-28T00:00:00.000000000',
       '2014-03-31T00:00:00.000000000', '2014-04-30

In [70]:
rebalancing_dates

array(['2013-01-31T00:00:00.000000000', '2013-02-28T00:00:00.000000000',
       '2013-03-29T00:00:00.000000000', '2013-04-30T00:00:00.000000000',
       '2013-05-31T00:00:00.000000000', '2013-06-28T00:00:00.000000000',
       '2013-07-31T00:00:00.000000000', '2013-08-30T00:00:00.000000000',
       '2013-09-30T00:00:00.000000000', '2013-10-31T00:00:00.000000000',
       '2013-11-29T00:00:00.000000000', '2013-12-31T00:00:00.000000000',
       '2014-01-31T00:00:00.000000000', '2014-02-28T00:00:00.000000000',
       '2014-03-31T00:00:00.000000000', '2014-04-30T00:00:00.000000000',
       '2014-05-30T00:00:00.000000000', '2014-06-30T00:00:00.000000000',
       '2014-07-31T00:00:00.000000000', '2014-08-29T00:00:00.000000000',
       '2014-09-30T00:00:00.000000000', '2014-10-31T00:00:00.000000000',
       '2014-11-28T00:00:00.000000000', '2014-12-31T00:00:00.000000000',
       '2015-01-30T00:00:00.000000000', '2015-02-27T00:00:00.000000000',
       '2015-03-31T00:00:00.000000000', '2015-04-30

In [241]:
filter_tails_percent = 0.20

train_size = 0.80
test_size = 0.20

backtest_results = pd.DataFrame(index=rebalancing_dates)
backtest_results['ML_Strategy_Return'] = np.nan
backtest_results['EQUAL_WT_Return'] = np.nan
backtest_results['Model_Accuracy'] = np.nan

for i in range(len(model_training_end_dates)-12):
    rebalance_date = rebalancing_dates[i]
    model_start_date = model_training_start_dates[i]
    model_end_date = model_training_end_dates[i]
    print("Processing data for "+str(rebalance_date))
    
    sample_data = stock_prices_data.loc[model_start_date:model_end_date]
    sample_data = sample_data.dropna()
    sample_data = sample_data.reset_index()
    
    training_data = sample_data.iloc[0:int(len(sample_data)*train_size)]
    test_data = sample_data.iloc[int(len(sample_data)*train_size)+1:]
    
    # Calculate Lower Threshold to filter samples at tails
    new_df = pd.DataFrame(training_data.loc[:,['Date', 'Forward_NormRet']].groupby('Date').quantile(filter_tails_percent)).rename(
        columns={'Forward_NormRet':'Lower_Threshold'})

    training_data = training_data.set_index(['Date']).join(new_df).reset_index()

    # Calculate Upper Threshold to filter samples at tails
    new_df = pd.DataFrame(training_data.loc[:,['Date', 'Forward_NormRet']].groupby('Date').quantile(1-filter_tails_percent)).rename(
        columns={'Forward_NormRet':'Upper_Threshold'})

    training_data = training_data.set_index(['Date']).join(new_df).reset_index()
    
    training_data_filtered = training_data[(training_data['Forward_NormRet']>=training_data['Upper_Threshold'])|(training_data['Forward_NormRet']<=training_data['Lower_Threshold'])]
    
    y_train = training_data_filtered['Target_Var']
    X_train = training_data_filtered[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
           '1MonthReturn','CCI_6M', 'CCI_12M', 'CCI_24M', 'CCI_36M', 'CCI_48M', 'CCI_60M',
           'RSI_6M', 'RSI_12M', 'RSI_24M', 'RSI_36M', 'RSI_48M', 'RSI_60M',
           'ROC_6M', 'ROC_12M', 'ROC_24M', 'ROC_36M', 'ROC_48M', 'ROC_60M',
           'STDDEV_6M', 'STDDEV_12M', 'STDDEV_24M', 'STDDEV_36M', 'STDDEV_48M',
           'STDDEV_60M', 'OBV']]
    
    model = MLModel(X_train,y_train,10,500)
    
    y_test = test_data['Target_Var']
    X_test = test_data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
           '1MonthReturn','CCI_6M', 'CCI_12M', 'CCI_24M', 'CCI_36M', 'CCI_48M', 'CCI_60M',
           'RSI_6M', 'RSI_12M', 'RSI_24M', 'RSI_36M', 'RSI_48M', 'RSI_60M',
           'ROC_6M', 'ROC_12M', 'ROC_24M', 'ROC_36M', 'ROC_48M', 'ROC_60M',
           'STDDEV_6M', 'STDDEV_12M', 'STDDEV_24M', 'STDDEV_36M', 'STDDEV_48M',
           'STDDEV_60M', 'OBV']]
    model_score = model.score(X_test, y_test)
    print("classification score %2f"%model_score)
    backtest_results.loc[rebalance_date,'Model_Accuracy'] = model_score
    
    rebalance_data = stock_prices_data.loc[rebalance_date].copy()
    original_data = stock_prices_data.loc[rebalance_date].copy()
    
    rebalance_data = rebalance_data.dropna()
    
    X_actual = rebalance_data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       '1MonthReturn','CCI_6M', 'CCI_12M', 'CCI_24M', 'CCI_36M', 'CCI_48M', 'CCI_60M',
       'RSI_6M', 'RSI_12M', 'RSI_24M', 'RSI_36M', 'RSI_48M', 'RSI_60M',
       'ROC_6M', 'ROC_12M', 'ROC_24M', 'ROC_36M', 'ROC_48M', 'ROC_60M',
       'STDDEV_6M', 'STDDEV_12M', 'STDDEV_24M', 'STDDEV_36M', 'STDDEV_48M',
       'STDDEV_60M', 'OBV']]
    
    rebalance_data = rebalance_data.assign(Outperform_Pred_Prob= pd.Series(model.predict_proba(X_actual)[:,1]).values)
    
    backtest_results.loc[rebalance_date,'ML_Strategy_Return'] = np.mean(rebalance_data.sort_values(['Outperform_Pred_Prob'], ascending=False).head(top_N)['1MonthReturn'])
    backtest_results.loc[rebalance_date,'EQUAL_WT_Return'] = np.mean(original_data['1MonthReturn'])

Processing data for 2013-01-31T00:00:00.000000000
classification score 0.720000
Processing data for 2013-02-28T00:00:00.000000000
classification score 0.640000
Processing data for 2013-03-29T00:00:00.000000000
classification score 0.710526
Processing data for 2013-04-30T00:00:00.000000000
classification score 0.764706
Processing data for 2013-05-31T00:00:00.000000000
classification score 0.710938
Processing data for 2013-06-28T00:00:00.000000000
classification score 0.677419
Processing data for 2013-07-31T00:00:00.000000000
classification score 0.668508
Processing data for 2013-08-30T00:00:00.000000000
classification score 0.764423
Processing data for 2013-09-30T00:00:00.000000000
classification score 0.770213
Processing data for 2013-10-31T00:00:00.000000000
classification score 0.683206
Processing data for 2013-11-29T00:00:00.000000000
classification score 0.678201
Processing data for 2013-12-31T00:00:00.000000000
classification score 0.670886
Processing data for 2014-01-31T00:00:00.

In [263]:
# Use the latest Available model for remaining dates
for i in range(-12,0):
    rebalance_date = rebalancing_dates[i]

    print("Processing data for "+str(rebalance_date))
    
    rebalance_data = stock_prices_data.loc[rebalance_date].copy()
    original_data = stock_prices_data.loc[rebalance_date].copy()
    del rebalance_data['Forward_NormRet']
    del rebalance_data['Forward_Holding_PeriodReturn']
    del rebalance_data['Forward_Std']
    del rebalance_data['MedianReturn']
    del rebalance_data['Target_Var']
    
    rebalance_data = rebalance_data.dropna()

    X_actual = rebalance_data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       '1MonthReturn','CCI_6M', 'CCI_12M', 'CCI_24M', 'CCI_36M', 'CCI_48M', 'CCI_60M',
       'RSI_6M', 'RSI_12M', 'RSI_24M', 'RSI_36M', 'RSI_48M', 'RSI_60M',
       'ROC_6M', 'ROC_12M', 'ROC_24M', 'ROC_36M', 'ROC_48M', 'ROC_60M',
       'STDDEV_6M', 'STDDEV_12M', 'STDDEV_24M', 'STDDEV_36M', 'STDDEV_48M',
       'STDDEV_60M', 'OBV']]
    
    rebalance_data = rebalance_data.assign(Outperform_Pred_Prob= pd.Series(model.predict_proba(X_actual)[:,1]).values)
    
    backtest_results.loc[rebalance_date,'ML_Strategy_Return'] = np.mean(rebalance_data.sort_values(['Outperform_Pred_Prob'], ascending=False).head(top_N)['1MonthReturn'])
    backtest_results.loc[rebalance_date,'EQUAL_WT_Return'] = np.mean(original_data['1MonthReturn'])

Processing data for 2020-01-31T00:00:00.000000000
Processing data for 2020-02-28T00:00:00.000000000
Processing data for 2020-03-31T00:00:00.000000000
Processing data for 2020-04-30T00:00:00.000000000
Processing data for 2020-05-29T00:00:00.000000000
Processing data for 2020-06-30T00:00:00.000000000
Processing data for 2020-07-31T00:00:00.000000000
Processing data for 2020-08-31T00:00:00.000000000
Processing data for 2020-09-30T00:00:00.000000000
Processing data for 2020-10-30T00:00:00.000000000
Processing data for 2020-11-30T00:00:00.000000000
Processing data for 2020-12-31T00:00:00.000000000


In [264]:
backtest_results.to_excel('backtest_results_v2_15Jan.xlsx')